<a href="https://colab.research.google.com/github/w-oke/covid_reproduction/blob/main/covid_owid_1_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pickle

The Our World In Data (OWID) project provides COVID-19 data on its websites at https://ourworldindata.org/coronavirus and https://github.com/owid/covid-19-data/tree/master/public/data


In [ ]:
# column descriptions:
owid_col_desc_link = 'https://github.com/owid/covid-19-data/raw/master/public/data/owid-covid-codebook.csv'
owid_col_desc = pd.read_csv(owid_col_desc_link)
owid_col_desc.head()

,column,source,category,description
0,iso_code,International Organization for Standardization,Others,ISO 3166-1 alpha-3 – three-letter country codes
1,continent,Our World in Data,Others,Continent of the geographical location
2,location,Our World in Data,Others,Geographical location
3,date,Our World in Data,Others,Date of observation
4,total_cases,COVID-19 Data Repository by the Center for Sys...,Confirmed cases,Total confirmed cases of COVID-19


In [ ]:
# create a list of all the columns in the dataset
cols = owid_col_desc.column.to_list()

In [ ]:
# possible independent variables
var = {'y': [ 
 # 'new_cases_smoothed_per_million',
 'reproduction_rate',
 # 'positive_rate',
 # 'tests_per_case', # inverse of positiviy rate
 ]}

# metadata - not intended for training
var['meta'] = [    
 'date',
 'iso_code',
 'location',
 # 'population', 
  ]

var['number'] = [
 # 'tests_units',
 'new_tests_smoothed_per_thousand',
 # 'total_vaccinations',
 # 'total_vaccinations_per_hundred', # use people_vaccinated & people_fully_vaccinated instead
 'people_vaccinated_per_hundred',
 'people_fully_vaccinated_per_hundred',
 'total_boosters_per_hundred',
 'stringency_index',
 'population_density',
 'median_age',
 'human_development_index',
 #'aged_65_older',
 #'aged_70_older',
 'gdp_per_capita',
 'extreme_poverty',
 #'cardiovasc_death_rate',
 #'diabetes_prevalence',
 #'female_smokers',
 #'male_smokers',
 'handwashing_facilities', # Share of the population with basic handwashing facilities on premises
 'hospital_beds_per_thousand', # Hospital beds per 1,000 people, most recent year available since 2010
 'life_expectancy',
 ]

In [ ]:
# create a single list of all the features
var_all = [item for sublist in list(var.values()) for item in sublist]
print('The first 4 items in "var_all" are: ', var_all[0:4])
print('There are {} variables in var_all'.format(len(var_all)))

# create a single string of all the features
var_all2 = ', '.join(var_all)
print('var_all2: ', var_all2)

The first 4 items in "var_all" are:  ['reproduction_rate', 'date', 'iso_code', 'location']
There are 17 variables in var_all
var_all2:  reproduction_rate, date, iso_code, location, new_tests_smoothed_per_thousand, people_vaccinated_per_hundred, people_fully_vaccinated_per_hundred, total_boosters_per_hundred, stringency_index, population_density, median_age, human_development_index, gdp_per_capita, extreme_poverty, handwashing_facilities, hospital_beds_per_thousand, life_expectancy


In [ ]:
# owid dataset:
owid_link = 'https://github.com/owid/covid-19-data/raw/master/public/data/owid-covid-data.csv'
owid = pd.read_csv(owid_link, usecols=var_all)
owid.dropna(subset=['reproduction_rate'], inplace=True)
owid['date'] = pd.to_datetime(owid['date']) # Convert the 'date' column to a Datetime format
owid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107755 entries, 34 to 136247
Data columns (total 17 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   iso_code                             107755 non-null  object        
 1   location                             107755 non-null  object        
 2   date                                 107755 non-null  datetime64[ns]
 3   reproduction_rate                    107755 non-null  float64       
 4   new_tests_smoothed_per_thousand      65863 non-null   float64       
 5   people_vaccinated_per_hundred        27707 non-null   float64       
 6   people_fully_vaccinated_per_hundred  25016 non-null   float64       
 7   total_boosters_per_hundred           5413 non-null    float64       
 8   stringency_index                     99508 non-null   float64       
 9   population_density                   106023 non-null  float64       


Note that:
* about half the data has 'handwashing_facilities' data
* about a quarter of the data has vaccination data, but this might be due to no vaccinations being available for large portions of the time


In [ ]:
owid.head()

,iso_code,location,date,reproduction_rate,new_tests_smoothed_per_thousand,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,stringency_index,population_density,median_age,gdp_per_capita,extreme_poverty,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
34,AFG,Afghanistan,2020-03-29,1.51,NaN,NaN,NaN,NaN,67.59,54.422,18.6,1803.987,NaN,37.746,0.5,64.83,0.511
35,AFG,Afghanistan,2020-03-30,1.51,NaN,NaN,NaN,NaN,67.59,54.422,18.6,1803.987,NaN,37.746,0.5,64.83,0.511
36,AFG,Afghanistan,2020-03-31,1.52,NaN,NaN,NaN,NaN,67.59,54.422,18.6,1803.987,NaN,37.746,0.5,64.83,0.511
37,AFG,Afghanistan,2020-04-01,1.51,NaN,NaN,NaN,NaN,67.59,54.422,18.6,1803.987,NaN,37.746,0.5,64.83,0.511
38,AFG,Afghanistan,2020-04-02,1.51,NaN,NaN,NaN,NaN,67.59,54.422,18.6,1803.987,NaN,37.746,0.5,64.83,0.511


## Variant Data

The OWID variant data is provided by country at a frequency of every two weeks.

In [ ]:
variant_link = 'https://github.com/owid/covid-19-data/raw/master/public/data/variants/covid-variants.csv'
variants = pd.read_csv(variant_link)
variants['date'] = pd.to_datetime(variants['date']) # Convert the 'date' column to a Datetime format
variants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54096 entries, 0 to 54095
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   location             54096 non-null  object        
 1   date                 54096 non-null  datetime64[ns]
 2   variant              54096 non-null  object        
 3   num_sequences        54096 non-null  int64         
 4   perc_sequences       54096 non-null  float64       
 5   num_sequences_total  54096 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 2.5+ MB


In [ ]:
variants.head()

,location,date,variant,num_sequences,perc_sequences,num_sequences_total
0,Angola,2020-12-21,Alpha,0,0.0,93
1,Angola,2020-12-21,B.1.1.277,0,0.0,93
2,Angola,2020-12-21,B.1.1.302,0,0.0,93
3,Angola,2020-12-21,B.1.1.519,0,0.0,93
4,Angola,2020-12-21,B.1.160,0,0.0,93


In [ ]:
variants.drop(['num_sequences', 'num_sequences_total'], axis=1, inplace=True)
variants.variant.unique()

array(['Alpha', 'B.1.1.277', 'B.1.1.302', 'B.1.1.519', 'B.1.160',
       'B.1.177', 'B.1.221', 'B.1.258', 'B.1.367', 'B.1.620', 'Beta',
       'Delta', 'Epsilon', 'Eta', 'Gamma', 'Iota', 'Kappa', 'Lambda',
       'Mu', 'Omicron', 'S:677H.Robin1', 'S:677P.Pelican', 'others',
       'non_who'], dtype=object)

In [ ]:
# create a list of the variants of interest
var['variants'] = '''
    Alpha
    Beta
    Delta
    Epsilon
    Eta
    Gamma
    Iota
    Kappa
    Lambda
    Mu
    Omicron
    non_who'''.split()
# note that the sum will add to 100% for any given day/location

In [ ]:
variants2 = variants[variants.variant.isin(var['variants'])]
variants2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27048 entries, 0 to 54095
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   location        27048 non-null  object        
 1   date            27048 non-null  datetime64[ns]
 2   variant         27048 non-null  object        
 3   perc_sequences  27048 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.0+ MB


In [ ]:
variants3 = variants2.pivot(index=['location','date'], columns='variant', values='perc_sequences')
variants3.reset_index(inplace=True)
variants3

variant,location,date,Alpha,Beta,Delta,Epsilon,Eta,Gamma,Iota,Kappa,Lambda,Mu,Omicron,non_who
0,Angola,2020-12-21,0.00,74.19,0.0,0.0,1.08,0.0,0.0,0.0,0.0,0.0,0.0,24.73
1,Angola,2021-01-25,5.77,3.85,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,90.38
2,Angola,2021-02-08,9.52,28.57,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,61.91
3,Angola,2021-02-22,2.56,41.03,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,56.41
4,Angola,2021-03-08,6.19,61.06,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,32.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,Zimbabwe,2021-01-11,0.00,95.61,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,4.39
2250,Zimbabwe,2021-01-25,0.00,94.74,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,5.26
2251,Zimbabwe,2021-02-08,0.00,100.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2252,Zimbabwe,2021-07-12,0.00,0.00,100.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [ ]:
variants3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2254 entries, 0 to 2253
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   location  2254 non-null   object        
 1   date      2254 non-null   datetime64[ns]
 2   Alpha     2254 non-null   float64       
 3   Beta      2254 non-null   float64       
 4   Delta     2254 non-null   float64       
 5   Epsilon   2254 non-null   float64       
 6   Eta       2254 non-null   float64       
 7   Gamma     2254 non-null   float64       
 8   Iota      2254 non-null   float64       
 9   Kappa     2254 non-null   float64       
 10  Lambda    2254 non-null   float64       
 11  Mu        2254 non-null   float64       
 12  Omicron   2254 non-null   float64       
 13  non_who   2254 non-null   float64       
dtypes: datetime64[ns](1), float64(12), object(1)
memory usage: 246.7+ KB


We want to increase the amount of data available.  We will do this by:
1. Populating dates between each 2-week date stamp.  The values will be the average of the previous and following week's data.
2. Creating a 'Global' location for each week who's values will be the average of all available locations for that week.

In [ ]:
dates_new = variants3.date[:-1] + pd.Timedelta('1W')
dates_new

0      2020-12-28
1      2021-02-01
2      2021-02-15
3      2021-03-01
4      2021-03-15
          ...    
2248   2021-01-11
2249   2021-01-18
2250   2021-02-01
2251   2021-02-15
2252   2021-07-19
Name: date, Length: 2253, dtype: datetime64[ns]

In [ ]:
variants3.date[0]+pd.Timedelta('1W')

Timestamp('2020-12-28 00:00:00')

In [ ]:
# create a new dataframe based on the existing one, which will be populated with the dates between each 2-week datestamp. The values will be the average of the previous and following week's data.
def edit_week(df, row1, row2, cols):
    ''' If the two 2nd row is two weeks after the first row in the same location,
    modify the first row so that its date is between the two dates and the variant data
    contains the average values of the two rows (for the columns in cols).
    Note: The df must be sorted by Location and Date!

    df: the dataframe that will be modified
    row1: the name of the row to be modified
    row2: the name of the 2nd row to be checked/averaged
    cols: a list of the columns to be averaged'''
    if (df.location[row1] == df.location[row2]) and (df.date[row2] == df.date[row1]+pd.Timedelta('2W')):
        df.loc[row1, 'date'] = df.loc[row1, 'date'] + pd.Timedelta('1W')
        df.loc[row1,cols] = (df.loc[row1,cols] + df.loc[row2,cols])/2
    else:
        df.drop(row1, inplace=True)
    return df

In [ ]:
variants3W = variants3.copy()
cols = variants3.columns[2:].to_list() # identify the columns that have the variant data
idx = variants3.index
for n in range(len(idx)-1):
    variants3W = edit_week(variants3W, idx[n], idx[n+1], cols)
variants3W.drop(idx[-1], inplace=True) # drop the last row

In [ ]:
variants3W.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1902 entries, 1 to 2252
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   location  1902 non-null   object        
 1   date      1902 non-null   datetime64[ns]
 2   Alpha     1902 non-null   float64       
 3   Beta      1902 non-null   float64       
 4   Delta     1902 non-null   float64       
 5   Epsilon   1902 non-null   float64       
 6   Eta       1902 non-null   float64       
 7   Gamma     1902 non-null   float64       
 8   Iota      1902 non-null   float64       
 9   Kappa     1902 non-null   float64       
 10  Lambda    1902 non-null   float64       
 11  Mu        1902 non-null   float64       
 12  Omicron   1902 non-null   float64       
 13  non_who   1902 non-null   float64       
dtypes: datetime64[ns](1), float64(12), object(1)
memory usage: 222.9+ KB


In [ ]:
variants4 = variants3.append(variants3W)
variants4.sort_values(by=['location','date'], inplace=True)
variants4.reset_index(drop=True, inplace=True)
variants4.head()

variant,location,date,Alpha,Beta,Delta,Epsilon,Eta,Gamma,Iota,Kappa,Lambda,Mu,Omicron,non_who
0,Angola,2020-12-21,0.000,74.19,0.0,0.0,1.08,0.0,0.0,0.0,0.0,0.0,0.0,24.730
1,Angola,2021-01-25,5.770,3.85,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,90.380
2,Angola,2021-02-01,7.645,16.21,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,76.145
3,Angola,2021-02-08,9.520,28.57,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,61.910
4,Angola,2021-02-15,6.040,34.80,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,59.160


In [ ]:
variants4.shape

(4156, 14)

### Create a 'Global' location
With the average of all the countries data.

In [ ]:
variants_glob = variants4.groupby('date').mean()
variants_glob['date'] = variants_glob.index
variants_glob['location'] = 'Global'
variants_glob.reset_index(drop=True, inplace=True)
variants_glob.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Alpha     81 non-null     float64       
 1   Beta      81 non-null     float64       
 2   Delta     81 non-null     float64       
 3   Epsilon   81 non-null     float64       
 4   Eta       81 non-null     float64       
 5   Gamma     81 non-null     float64       
 6   Iota      81 non-null     float64       
 7   Kappa     81 non-null     float64       
 8   Lambda    81 non-null     float64       
 9   Mu        81 non-null     float64       
 10  Omicron   81 non-null     float64       
 11  non_who   81 non-null     float64       
 12  date      81 non-null     datetime64[ns]
 13  location  81 non-null     object        
dtypes: datetime64[ns](1), float64(12), object(1)
memory usage: 9.0+ KB


## Merge the OWID and Variant data:

In [ ]:
owid2 = pd.merge(left=owid, right=variants_glob,
    how="inner",
    on=['date'],
    suffixes=[None, "_y"]
)
owid2.drop('location_y', axis=1, inplace=True)

In [ ]:
owid2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14301 entries, 0 to 14300
Data columns (total 29 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   iso_code                             14301 non-null  object        
 1   location                             14301 non-null  object        
 2   date                                 14301 non-null  datetime64[ns]
 3   reproduction_rate                    14301 non-null  float64       
 4   new_tests_smoothed_per_thousand      8803 non-null   float64       
 5   people_vaccinated_per_hundred        4093 non-null   float64       
 6   people_fully_vaccinated_per_hundred  3690 non-null   float64       
 7   total_boosters_per_hundred           738 non-null    float64       
 8   stringency_index                     13289 non-null  float64       
 9   population_density                   14064 non-null  float64       
 10  median_age

In [ ]:
owid_countries = pd.merge(left=owid, right=variants4,
    how="inner",
    on=['location', 'date']
)
owid_countries.shape

(4027, 29)

In [ ]:
# we want to replace the Global data with  the Countries data where available.
# To do so, we will add the two datasets together, and then drop the duplicate
# entries (that match both location & date) from merged set, keeping the Country data
owid3 = owid_countries.append(owid2)
owid3.shape

(18328, 29)

In [ ]:
owid3.drop_duplicates(subset=['location', 'date'], inplace=True, keep='first', ignore_index=True)
owid3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14301 entries, 0 to 14300
Data columns (total 29 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   iso_code                             14301 non-null  object        
 1   location                             14301 non-null  object        
 2   date                                 14301 non-null  datetime64[ns]
 3   reproduction_rate                    14301 non-null  float64       
 4   new_tests_smoothed_per_thousand      8803 non-null   float64       
 5   people_vaccinated_per_hundred        4093 non-null   float64       
 6   people_fully_vaccinated_per_hundred  3690 non-null   float64       
 7   total_boosters_per_hundred           738 non-null    float64       
 8   stringency_index                     13289 non-null  float64       
 9   population_density                   14064 non-null  float64       
 10  median_age

In [ ]:
owid3.to_parquet('covid_owid_df.parquet') # output to a parquet file

In [ ]:
var

{'meta': ['date', 'iso_code', 'location'],
 'number': ['new_tests_smoothed_per_thousand',
  'people_vaccinated_per_hundred',
  'people_fully_vaccinated_per_hundred',
  'total_boosters_per_hundred',
  'stringency_index',
  'population_density',
  'median_age',
  'human_development_index',
  'gdp_per_capita',
  'extreme_poverty',
  'handwashing_facilities',
  'hospital_beds_per_thousand',
  'life_expectancy'],
 'variants': ['Alpha',
  'Beta',
  'Delta',
  'Epsilon',
  'Eta',
  'Gamma',
  'Iota',
  'Kappa',
  'Lambda',
  'Mu',
  'Omicron',
  'non_who'],
 'y': ['reproduction_rate']}

In [ ]:
# save the variables to file
with open('covid_owid_var_dictionary.pkl', 'wb') as f:
    pickle.dump(var, f)